In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torchvision
from torchvision import models
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA
from sklearn.covariance import LedoitWolf, MinCovDet

from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score

np.random.seed(252525)
torch.manual_seed(252525)

import torch
import torch.nn as nn

from meanshift import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
net = models.efficientnet_b4(pretrained=True).features
net = net.to(device)
net = net.eval()
device

'cuda'

In [8]:
bs = 50
clazz = 3
epochs = 1
augment = False

log = []

for run in range(2):

    for clazz in range(10):

        X_, X_valid_, X_test_ = Cifar10_OneClass(train_classes=[clazz], balance=False, fold=run, transform=transforms.Compose([transforms.ToTensor()]))[0]
        X_ = X_.reshape(len(X_), 3, 32, 32)
        X_valid_ = X_valid_.reshape((len(X_valid_), 3, 32, 32))
        X_test_ = X_test_.reshape((len(X_test_), 3, 32, 32))

        scores_inliers = []
        scores_outliers = []

        for l in range(9):

            X = concat_features(X_, net, layers=[l], fmap_pool=False)
            X_valid = concat_features(X_valid_, net, layers=[l], fmap_pool=False)
            X_test = concat_features(X_test_, net, layers=[l], fmap_pool=False)

            # Pooling training
            n = X.shape[0]
            X__ = X.mean((2,3))
            #X__ = X__.reshape(n, -1)

            C_inv = LedoitWolf().fit(X__).precision_
            mu = X__.mean(0)

            # Pooling inliers
            n_valid = len(X_valid)
            X_valid__ = X_valid.mean((2,3)) - mu
            #X_valid_ = X_valid_.reshape(n_valid, -1)

            # Pooling outliers
            n_test = len(X_test)
            X_test__ = X_test.mean((2,3)) - mu
            #X_test_ = X_test_.reshape(n_test, -1)

            # Mahalanobis
            M_valid_ = np.sqrt( ( X_valid__ * ( C_inv @ X_valid__.T ).T ).sum(1) )    
            M_test_  = np.sqrt( ( X_test__ * ( C_inv @ X_test__.T ).T ).sum(1) )    

            scores_inliers.append(M_valid_)
            scores_outliers.append(M_test_)

        auc = roc_auc_score([0] * n_valid + [1] * n_test, np.concatenate([np.asarray(scores_inliers).sum(0), np.asarray(scores_outliers).sum(0)]))

        log.append(pd.DataFrame(np.asarray([run, clazz, l, n, auc])[:, None].T, columns=["run", "class", "layers", "n_samples", "auc"])) 

        print(Cifar10_OneClass.CLASSES[clazz], auc)

Files already downloaded and verified
Files already downloaded and verified
plane 0.749074
Files already downloaded and verified
Files already downloaded and verified
car 0.7530600000000001
Files already downloaded and verified
Files already downloaded and verified
bird 0.635725
Files already downloaded and verified
Files already downloaded and verified
cat 0.650938
Files already downloaded and verified
Files already downloaded and verified
deer 0.737636
Files already downloaded and verified
Files already downloaded and verified
dog 0.703292
Files already downloaded and verified
Files already downloaded and verified
frog 0.7683699999999999
Files already downloaded and verified
Files already downloaded and verified
horse 0.765567
Files already downloaded and verified
Files already downloaded and verified
ship 0.7434560000000001
Files already downloaded and verified
Files already downloaded and verified
truck 0.7833589999999999
Files already downloaded and verified
Files already download

In [10]:
# GLOBAL layer 5/6
log_ = pd.concat(log)

for clazz in range(10):
    auc = log_[(log_["class"] == clazz)].groupby("run").mean()["auc"]
    print(f"class={Cifar10_OneClass.CLASSES[clazz]:10s}", f"\tauc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}")
    
log_ = pd.concat(log)

auc = log_.groupby("run").mean()["auc"]
print(f"avg.-auc: {auc.mean():.3f}", "\u00B1", f"{auc.std():.3f}")

class=plane      	auc: 0.745 ± 0.006
class=car        	auc: 0.748 ± 0.008
class=bird       	auc: 0.630 ± 0.008
class=cat        	auc: 0.657 ± 0.008
class=deer       	auc: 0.737 ± 0.002
class=dog        	auc: 0.706 ± 0.004
class=frog       	auc: 0.766 ± 0.004
class=horse      	auc: 0.757 ± 0.012
class=ship       	auc: 0.744 ± 0.000
class=truck      	auc: 0.779 ± 0.006
avg.-auc: 0.727 ± 0.003
